In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load Equinox
equinox = pd.read_excel('Equinox_on_GitHub_June9_2022.xlsx', 
                        sheet_name='Equinox2020_CanonDat')

# Filter to social complexity
sc_data = equinox[equinox['Section'] == 'Social Complexity variables']

# Extract 4 hierarchy variables
hierarchy_vars = [
    'Settlement hierarchy',
    'Administrative levels', 
    'Military levels',
    'Religious levels'
]

# Pivot long → wide format
hierarchy_df = sc_data[sc_data['Variable'].isin(hierarchy_vars)].pivot_table(
    index='Polity',
    columns='Variable',
    values='Value.From',
    aggfunc='first'  # Take first value if multiple time points
)

# Convert to numeric
for col in hierarchy_df.columns:
    hierarchy_df[col] = pd.to_numeric(hierarchy_df[col], errors='coerce')

# Filter: require all 4 variables
hierarchy_df = hierarchy_df.dropna()

print(f"Polities with all 4 hierarchy variables: {len(hierarchy_df)}")
print(f"2017 dataset had: 114")
print(f"Increase: {100*(len(hierarchy_df)-114)/114:.1f}%")


FileNotFoundError: [Errno 2] No such file or directory: 'Equinox_on_GitHub_June9_2022.xlsx'